In [2]:
from sympy import mod_inverse

In [3]:
p = 5
a = 2
b = 3

In [4]:
class ECPoint:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __eq__(self, other):
        return isinstance(other, ECPoint) and self.x == other.x and self.y == other.y

    def __neg__(self):
        return ECPoint(self.x, (-self.y) % p)

    def is_infinity(self):
        return self.x is None and self.y is None

    def __repr__ (self):
        if self.is_infinity():
            return "O"
        return f"({self.x}, {self.y})"

In [5]:
O = ECPoint(None, None)

In [6]:
def ec_add(P, Q):
    if P.is_infinity():
        return Q
    if Q.is_infinity():
        return P
    if Q == -P:
        return O
    if P != Q:
        try:
            l = ((Q.y - P.y) * mod_inverse(Q.x - P.x, p)) % p
        except ValueError:
            return O
    else:
        if P.y == 0:
            return O
        l = ((3 * P.x**2 + a) * mod_inverse(2 * P.y, p)) % p

    x_r = (l**2 - P.x - Q.x) % p
    y_r = (l * (P.x - x_r) - P.y) % p
    return ECPoint(x_r, y_r)


In [7]:
def ec_mul(P, n):
    R = O
    for i in range(n):
        R = ec_add(R, P)
    return R

In [21]:
def list_points():
    points = [O]
    for x in range(p):
        rhs = (x**3 + a*x + b) % p
        for y in range(p):
            if (y*y) % p == rhs:
                points.append(ECPoint(x, y))
    return points

In [22]:
list_points()

[O, (1, 1), (1, 4), (2, 0), (3, 1), (3, 4), (4, 0)]

In [10]:
G = ECPoint(1, 1)
for i in range(7):
    print(ec_mul(G, i))

O
(1, 1)
(3, 4)
(2, 0)
(3, 1)
(1, 4)
O


In [11]:
k = 3
P = ec_mul(G, 3)

In [16]:

mat = list(list(range(8)) for _ in range(8))
for i in range(8):
    for j in range(8):
        mat[i][j] = ec_add(ec_mul(P, i), ec_mul(G, j))
mat

[[O, (1, 1), (3, 4), (2, 0), (3, 1), (1, 4), O, (1, 1)],
 [(2, 0), (3, 1), (1, 4), O, (1, 1), (3, 4), (2, 0), (3, 1)],
 [O, (1, 1), (3, 4), (2, 0), (3, 1), (1, 4), O, (1, 1)],
 [(2, 0), (3, 1), (1, 4), O, (1, 1), (3, 4), (2, 0), (3, 1)],
 [O, (1, 1), (3, 4), (2, 0), (3, 1), (1, 4), O, (1, 1)],
 [(2, 0), (3, 1), (1, 4), O, (1, 1), (3, 4), (2, 0), (3, 1)],
 [O, (1, 1), (3, 4), (2, 0), (3, 1), (1, 4), O, (1, 1)],
 [(2, 0), (3, 1), (1, 4), O, (1, 1), (3, 4), (2, 0), (3, 1)]]

In [20]:
# 行列からすべての(3,1)のインデックスを取得
target_point = O
indices = []

for i in range(7):
    for j in range(7):
        if mat[i][j] == target_point:
            indices.append((i, j))

print(f"(3,1)が見つかったインデックス: {indices}")
print(f"合計{len(indices)}個見つかりました")

(3,1)が見つかったインデックス: [(0, 0), (0, 6), (1, 3), (2, 0), (2, 6), (3, 3), (4, 0), (4, 6), (5, 3), (6, 0), (6, 6)]
合計11個見つかりました


In [18]:
import cmath
import math
import numpy as np

# 定数
N = 8
coeff = 1 / (8 * math.sqrt(7))

# 各 (j, k) ペアの振幅を格納する行列
amplitudes = np.zeros((N, N), dtype=complex)

print("Calculating Amplitudes A_jk:")
print("----------------------------")

for j in range(N):
    for k in range(N):
        # 各指数関数項を計算
        term1 = cmath.exp(-1j * math.pi * j)
        term2 = cmath.exp(-1j * math.pi / 4 * (j + k))
        term3 = cmath.exp(-1j * math.pi / 2 * (j + 2 * k))
        term4 = cmath.exp(-1j * math.pi / 4 * (3 * j + k))
        term5 = cmath.exp(-1j * math.pi * j - 1j * math.pi / 4 * k)
        term6 = cmath.exp(-1j * math.pi / 4 * (5 * j + k))
        term7 = cmath.exp(-1j * 3 * math.pi / 2 * j - 1j * math.pi * k)

        # 7つの項の合計
        sum_of_terms = term1 + term2 + term3 + term4 + term5 + term6 + term7

        # 振幅を計算
        amplitudes[j, k] = coeff * sum_of_terms

        # 振幅の絶対値を計算 (測定確率の平方根)
        abs_amplitude = abs(amplitudes[j, k])

        # 小数点以下3桁で表示
        print(f"A_{j}{k}: {amplitudes[j, k]:.3f} (Magnitude: {abs_amplitude:.3f})")

# すべての振幅の絶対値の分布を表示
print("\nMagnitude of Amplitudes (abs(A_jk)) Matrix:")
print("------------------------------------------")
# 表示を見やすくするため、絶対値のみを3桁で表示
abs_amplitudes_matrix = np.array([[abs(val) for val in row] for row in amplitudes])
print(np.round(abs_amplitudes_matrix, 3))

# 振幅の絶対値の最大値と最小値、およびそれらの差を確認
max_amp = np.max(abs_amplitudes_matrix)
min_amp = np.min(abs_amplitudes_matrix)
print(f"\nMaximum Amplitude Magnitude: {max_amp:.3f}")
print(f"Minimum Amplitude Magnitude: {min_amp:.3f}")
print(f"Difference: {max_amp - min_amp:.3f}")

# 振幅の絶対値が特定の閾値（例えば最大値の半分）より大きいペアを特定
threshold = max_amp * 0.5 # 例として最大振幅の半分を閾値とする
high_amplitude_pairs = []
for j in range(N):
    for k in range(N):
        if abs(amplitudes[j, k]) > threshold:
            high_amplitude_pairs.append((j, k, abs(amplitudes[j, k])))

if high_amplitude_pairs:
    print(f"\nPairs with Amplitude Magnitude > {threshold:.3f} (High Probability States):")
    for pair in high_amplitude_pairs:
        print(f"  (j={pair[0]}, k={pair[1]}): Magnitude {pair[2]:.3f}")
else:
    print("\nNo high amplitude pairs found above the threshold. Amplitudes might be relatively uniform.")

Calculating Amplitudes A_jk:
----------------------------
A_00: 0.331+0.000j (Magnitude: 0.331)
A_01: 0.086-0.134j (Magnitude: 0.159)
A_02: 0.142-0.189j (Magnitude: 0.236)
A_03: -0.181-0.134j (Magnitude: 0.225)
A_04: -0.047+0.000j (Magnitude: 0.047)
A_05: -0.181+0.134j (Magnitude: 0.225)
A_06: 0.142+0.189j (Magnitude: 0.236)
A_07: 0.086+0.134j (Magnitude: 0.159)
A_10: -0.128-0.033j (Magnitude: 0.132)
A_11: -0.128+0.033j (Magnitude: 0.132)
A_12: -0.081+0.081j (Magnitude: 0.114)
A_13: -0.014+0.081j (Magnitude: 0.082)
A_14: 0.033+0.033j (Magnitude: 0.047)
A_15: 0.033-0.033j (Magnitude: 0.047)
A_16: -0.014-0.081j (Magnitude: 0.082)
A_17: -0.081-0.081j (Magnitude: 0.114)
A_20: 0.000-0.047j (Magnitude: 0.047)
A_21: 0.142-0.067j (Magnitude: 0.157)
A_22: -0.094-0.047j (Magnitude: 0.106)
A_23: 0.075+0.000j (Magnitude: 0.075)
A_24: -0.094+0.047j (Magnitude: 0.106)
A_25: 0.142+0.067j (Magnitude: 0.157)
A_26: -0.000+0.047j (Magnitude: 0.047)
A_27: 0.209+0.000j (Magnitude: 0.209)
A_30: -0.061-0.033